In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,TensorDataset
from gensim.models import Word2Vec
from models import CNNClassifier, RNNClassifier, GRUClassifier, LSTMClassifier, TClassifier
from config_dataset import custom_dataset
from accuracy import classifier_accuracy

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
sequence_length = 20
batch_size = 32
embedding_dim = 300
hidden_dim = 8

In [4]:
train_loader, val_loader, embedding_dim, embedding_matrix, word2vec, idx2word, word2idx, vocab_size, style0_test, style1_test, style2_test = custom_dataset('text_corpus/divina_commedia.txt',
                                                                                                                                                        'text_corpus/uno_nessuno_e_i_malavoglia.txt',
                                                                                                                                                        'text_corpus/lo_cunto_de_li_cunti.txt',
                                                                                                                                                        sequence_length,
                                                                                                                                                        embedding_dim,
                                                                                                                                                        batch_size,
                                                                                                                                                        0.8)
print('len train loader: ', len(train_loader))
print('len val loader: ', len(val_loader))
print('total number of training samples: ', len(train_loader.dataset))
print('total number of validation samples: ', len(val_loader.dataset))
print('vocab size: ', vocab_size)

len train loader:  633
len val loader:  80
total number of training samples:  20246
total number of validation samples:  2529
vocab size:  26606


# CNN Classifier

In [5]:
cnn_classifier = CNNClassifier(embedding_matrix, 3, 3, [3,3,3])

cnn_classifier.load_state_dict(torch.load('pretrained/cnn_classifier.pth'))

classifier_params = sum(p.numel() for p in cnn_classifier.parameters() if p.requires_grad)
print('# Parameters: ', classifier_params)

# Parameters:  8139


In [6]:
classifier_accuracy(cnn_classifier, style0_test, style1_test, style2_test, 'CNN')

Accuracy predicting Dante:  0.9852507374631269
Accuracy predicting Italian:  0.992133726647001
Accuracy predicting Neapolitan:  0.998
Overall  CNN  Accuracy:  0.9917948213700427


In [9]:
for i ,(data, label) in enumerate(val_loader):
    if i == 0:
        prova = data[0]
        labels = label[0]
    else: 
        break

input_sentence = [idx2word[prova[i].item()] for i in range(prova.shape[0])]

prova = prova.view(1,prova.shape[0])

with torch.no_grad():
    pred_label = cnn_classifier(prova)
    
pred_label = torch.softmax(pred_label, dim = -1)


if labels == 0.0:
    stile = 'Dante'
elif labels == 1.0:
    stile = 'Italiano'
elif labels == 2.0:
    stile = 'Napoletano'


pred_style = torch.argmax(pred_label).item()
print('Real Style: ', labels.item(), ', ', stile)
print('Predicted Style: ', pred_style, 'with probability: ', pred_label[0][0][pred_style].item()*100, '%')
print("Input sequence: \n", ' '.join(input_sentence))

Real Style:  1 ,  Italiano
Predicted Style:  1 with probability:  57.60539770126343 %
Input sequence: 
 e vedrai come andrà a finire vedrai che arriverà all orecchio del vicario e ti leveranno la medaglia di figlia


# RNN Classifier

In [7]:
rnn_classifier = RNNClassifier(embedding_matrix, hidden_dim, 1)

rnn_classifier.load_state_dict(torch.load('pretrained/rnn_classifier.pth'))

classifier_params = sum(p.numel() for p in rnn_classifier.parameters() if p.requires_grad)
print('# Parameters: ', classifier_params)

# Parameters:  3107


In [8]:
classifier_accuracy(rnn_classifier, style0_test, style1_test, style2_test, 'RNN')

Accuracy predicting Dante:  0.9823008849557522
Accuracy predicting Italian:  0.976401179941003
Accuracy predicting Neapolitan:  0.978
Overall  RNN  Accuracy:  0.9789006882989183


In [16]:
for i ,(data, label) in enumerate(val_loader):
    if i == 0:
        prova = data[0]
        labels = label[0]
    else: 
        break

input_sentence = [idx2word[prova[i].item()] for i in range(prova.shape[0])]

prova = prova.view(1,prova.shape[0])

with torch.no_grad():
    pred_label = rnn_classifier(prova)
    
pred_label = torch.softmax(pred_label, dim = -1)


if labels == 0.0:
    stile = 'Dante'
elif labels == 1.0:
    stile = 'Italiano'
elif labels == 2.0:
    stile = 'Napoletano'


pred_style = torch.argmax(pred_label).item()
print('Real Style: ', labels.item(), ', ', stile)
print('Predicted Style: ', pred_style, 'with probability: ', pred_label[0][0][pred_style].item()*100, '%')
print("Input sequence: \n", ' '.join(input_sentence))

Real Style:  1 ,  Italiano
Predicted Style:  2 with probability:  83.76763463020325 %
Input sequence: 
 fortuna altrove le poverette s intendevano fra di loro appunto per questo quando discorrevano a bassa voce col capo chino


# GRU Classifier

In [9]:
gru_classifier = GRUClassifier(embedding_matrix, hidden_dim, 1)

gru_classifier.load_state_dict(torch.load('pretrained/gru_classifier.pth'))

classifier_params = sum(p.numel() for p in gru_classifier.parameters() if p.requires_grad)
print('# Parameters: ', classifier_params)

# Parameters:  8067


In [10]:
classifier_accuracy(gru_classifier, style0_test, style1_test, style2_test, 'GRU')

Accuracy predicting Dante:  0.9754178957718781
Accuracy predicting Italian:  0.992133726647001
Accuracy predicting Neapolitan:  0.996
Overall  GRU  Accuracy:  0.987850540806293


In [15]:
for i ,(data, label) in enumerate(val_loader):
    if i == 0:
        prova = data[0]
        labels = label[0]
    else: 
        break

input_sentence = [idx2word[prova[i].item()] for i in range(prova.shape[0])]

prova = prova.view(1,prova.shape[0])

with torch.no_grad():
    pred_label = gru_classifier(prova)
    
pred_label = torch.softmax(pred_label, dim = -1)


if labels == 0.0:
    stile = 'Dante'
elif labels == 1.0:
    stile = 'Italiano'
elif labels == 2.0:
    stile = 'Napoletano'


pred_style = torch.argmax(pred_label).item()
print('Real Style: ', labels.item(), ', ', stile)
print('Predicted Style: ', pred_style, 'with probability: ', pred_label[0][0][pred_style].item()*100, '%')
print("Input sequence: \n", ' '.join(input_sentence))

Real Style:  1 ,  Italiano
Predicted Style:  1 with probability:  99.52223896980286 %
Input sequence: 
 cercare massaro filippo nella bottega di pizzuto tastando i muri col bastone sua figlia faceva la sdegnosa protestando che non


# LSTM Classifier

In [11]:
lstm_classifier = LSTMClassifier(embedding_matrix, hidden_dim, 1)

lstm_classifier.load_state_dict(torch.load('pretrained/lstm_classifier.pth'))

classifier_params = sum(p.numel() for p in lstm_classifier.parameters() if p.requires_grad)
print('# Parameters: ', classifier_params)

# Parameters:  10547


In [12]:
classifier_accuracy(lstm_classifier, style0_test, style1_test, style2_test, 'LSTM')

Accuracy predicting Dante:  0.983284169124877
Accuracy predicting Italian:  0.9950835791543756
Accuracy predicting Neapolitan:  0.996
Overall  LSTM  Accuracy:  0.9914559160930843


In [20]:
for i ,(data, label) in enumerate(val_loader):
    if i == 0:
        prova = data[0]
        labels = label[0]
    else: 
        break

input_sentence = [idx2word[prova[i].item()] for i in range(prova.shape[0])]

prova = prova.view(1,prova.shape[0])

with torch.no_grad():
    pred_label = lstm_classifier(prova)
    
pred_label = torch.softmax(pred_label, dim = -1)


if labels == 0.0:
    stile = 'Dante'
elif labels == 1.0:
    stile = 'Italiano'
elif labels == 2.0:
    stile = 'Napoletano'


pred_style = torch.argmax(pred_label).item()
print('Real Style: ', labels.item(), ', ', stile)
print('Predicted Style: ', pred_style, 'with probability: ', pred_label[0][0][pred_style].item()*100, '%')
print("Input sequence: \n", ' '.join(input_sentence))

Real Style:  2 ,  Napoletano
Predicted Style:  2 with probability:  99.6376097202301 %
Input sequence: 
 sta sera lo fatte festa sarraggio cannariato da mammata ed io non tanto averraggio tormiento de scrastareme da sto nigro


# Transformer Classifier

In [13]:
t_classifier = TClassifier(embedding_matrix)

t_classifier.load_state_dict(torch.load('pretrained/t_classifier.pth'))

classifier_params = sum(p.numel() for p in t_classifier.parameters() if p.requires_grad)
print('# Parameters: ', classifier_params)

# Parameters:  3188599


In [14]:
classifier_accuracy(t_classifier, style0_test, style1_test, style2_test, 'Transformer')

Accuracy predicting Dante:  0.984267453294002
Accuracy predicting Italian:  0.9950835791543756
Accuracy predicting Neapolitan:  1.0
Overall  Transformer  Accuracy:  0.9931170108161259


In [23]:
for i ,(data, label) in enumerate(val_loader):
    if i == 0:
        prova = data[0]
        labels = label[0]
    else: 
        break

input_sentence = [idx2word[prova[i].item()] for i in range(prova.shape[0])]

prova = prova.view(1,prova.shape[0])

with torch.no_grad():
    pred_label = t_classifier(prova)
    
pred_label = torch.softmax(pred_label, dim = -1)


if labels == 0.0:
    stile = 'Dante'
elif labels == 1.0:
    stile = 'Italiano'
elif labels == 2.0:
    stile = 'Napoletano'


pred_style = torch.argmax(pred_label).item()
print('Real Style: ', labels.item(), ', ', stile)
print('Predicted Style: ', pred_style, 'with probability: ', pred_label[0][0][pred_style].item()*100, '%')
print("Input sequence: \n", ' '.join(input_sentence))

Real Style:  0 ,  Dante
Predicted Style:  0 with probability:  99.9997615814209 %
Input sequence: 
 sì fermo che pur con cibi di liquor d ulivi lievemente passava caldi e geli contento ne pensier contemplativi render
